# Q-Learning en Machine Learning - Parte I

**SEGÚN FEDERICO:**

Es un tipo de algoritmo de aprendizaje por refuerzo que le permite a un agente imaginario aprender a tomar decisiones óptimas y a alcanzar un objetivo en ese entorno o ambiente determinado.

El agente opera **aprendiendo los valores que sean más convenientes para cada paso** que tenga que dar, y a esos pasos los implementamos en un par de datos que definen la acción que tiene que tomar y el estado en que queda.

Entonces, Q-Learning implica un agente que debe identificar qué pasos realizar y la **calidad de esos pasos** la vamos a denominar con la letra `Q`.

Los valores `Q`, es decir, los valores de sus pasos que se definen por los valores de la acción y del Estado, van a ayudar a este agente a decidir qué acción tiene que tomar en cada paso.

### Ecuación de Bellman para actualizar valores Q

![Bellman.png](./assets/Bellman.png)

Donde:

- `𝑄(𝑠,a)` = función 𝑄 que se está actualizando. Representa el valor actual estimado de tomar la acción 𝑎 en el estado 𝑠.

- `𝑠` = estado (state)

- `a` = acción (action)

- `α` = tasa de aprendizaje o **learning rate** (usualmente, un número entre 0 y 1). Controla qué tan rápido actualizamos la estimación del valor. Si α es cercano a 1, significa que confiamos mucho en las nuevas experiencias, y si es cercano a 0, confiamos más en los valores previos. Dicho de otra manera, según Federico:

    - Para un valor cercano a 1, el aprendizaje será más rápido, pero menos seguro, con estimaciones menos estables para los valores Q.
    
    - Para un valor cercano a 0, el aprendizaje será más lento, pero más seguro en llevar una estimación más estable de los valores Q.

- `R(𝑠,a)` = reforzador (reinforcer) o recompensa (reward), después de tomar la acción 𝑎 en el estado 𝑠. Esta recompensa indica el **beneficio obtenido** por realizar esa acción en ese momento específico.

- `γ` = factor de descuento o **discount factor**, también un número entre 0 y 1. Indica cuánto valoramos las recompensas futuras. Un valor cercano a 1 significa que nos importan mucho las recompensas futuras (lo suficiente para considerar las consecuencias a largo plazo de nuestras acciones), mientras que un valor cercano a 0 significa que solo nos interesa la recompensa inmediata.

- `𝑠′` = nuevo estado al que llega el agente después de tomar la acción 𝑎 en el estado 𝑠.

- `max a′ Q(𝑠′, a′)` = selecciona el valor máximo de la función 𝑄 sobre todas las acciones posibles 𝑎′ que el agente podría tomar en el nuevo estado 𝑠′. En otras palabras, el agente trata de estimar el mejor valor futuro posible en el siguiente estado, basándose en lo que ha aprendido hasta el momento.

### Caso de ejemplo

Nos imaginemos un robot que opera en una cuadrícula que representa el salón de una fábrica en la que debe moverse trasladando herramientas.

El objetivo es que el robot aprenda a encontrar el camino más corto, desde su posición inicial hasta el destino, evitando obstáculos.

Entonces, el entorno que vamos a definir para este ejercicio va a ser una cuadrícula de dimensiones de 5x5, un punto de inicio en la esquina superior izquierda, que sería el punto `(0, 0)`, el punto objetivo en la esquina inferior derecha, que sería el punto `(4, 4)` y los obstáculos distribuidos en la cuadrícula.

Vamos a establecer las acciones. Esto implica que el robot va a poder moverse en cuatro direcciones hacia arriba, hacia abajo, izquierda o derecha, y vamos a definir recompensas.

Si alcanza el objetivo, va a tener 100 puntos de premio.

Si colisiona con un obstáculo, se le van a restar 100 puntos y cualquier otro movimiento que haga va a valer -1.

¿Para que?

Para incentivar la eficiencia, que reste lo menos que pueda.

In [3]:
import random
import numpy as np

In [22]:
grid: tuple[int, int] = (5, 5)

initial_state: tuple[int, int] = (0, 0)
goal_state: tuple[int, int] = (4, 4)

obstacles: list[tuple[int, int]] = [(1, 1), (1, 3), (2, 3), (3, 0)]

actions: dict[str, tuple[int, int]] = {
    'up': (0, -1),
    'down': (0, 1),
    'left': (-1, 0),
    'right': (1, 0),
}

In [16]:
grid_states: int = grid[0] * grid[1]
grid_states

25

In [17]:
possible_actions: int = len(actions)

In [18]:
q_table: np.ndarray = np.zeros((grid_states, possible_actions))
q_table

array([[0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.]])

In [19]:
def convert_state_to_index(
        state: tuple[int, int],
        grid: tuple[int, int]
        ) -> int:
    """
    Convert the current two-dimensional representation of the current
    state (the agent's position on the grid) to an unique linear index.

    Every possible state can be represented as a index number for the
    Q table.

    Parameters
    ----------
    state : tuple[int, int]
        The current two-dimensional representation of the current state
        (the agent's position on the grid).

    grid : tuple[int, int]
        The width and height of the grid.

    Returns
    -------
    int
        The index of the current state in the Q table.
    
    Examples
    --------
    >>> convert_state_to_index((0, 0), (5, 5))
    0
    >>> convert_state_to_index((4, 4), (5, 5))
    24

    """
    return state[0] * grid[1] + state[1]

In [21]:
example = convert_state_to_index(initial_state, grid)
example

0

**¿Cómo el agente robot va a aprender a navegar por el entorno?**

Los siguientes parámetros son fundamentales:

In [23]:
alpha: float = 0.1      # learning rate
gamma: float = 0.99     # discount factor
epsilon: float = 0.2    # exploration
episodes: int = 100     # number of episodes

La variable `epsilon` sirve para que el agente no repita siempre las mismas acciones, definiendo la **probabilidad** de que el agente tome una **acción aleatoria** en lugar de la mejor acción conocida hasta el momento, según la tabla Q.

En otras palabras, esto permite que el agente **explore el entorno** desconocido en lugar de explotar constantemente el conocimiento que ya dispone, porque una vez que encuentre una solución, podría tender a repetir esa solución siempre, en vez de explorar otras posibilidades (que podrían ser más eficientes o de mayor interés, según sea el caso).

La variable `episodes` está definiendo el número total de veces (episodios) que se va a repetir este proceso de entrenamiento.

Un episodio comienza con el agente en el estado inicial (declarado en `initial_state`) y termina cuando alcanza el estado objetivo (declarado en `goal_state`) o algún otro criterio de finalización, según sea requerido.

Mientras mayor sea el número de episodios, más oportunidades va a tener el agente de tener más experiencias de las cuales aprender y esto va a mejorar potencialmente sus políticas de acción.